In [1]:
!pip install ta
!pip install pandas_ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=e78fb669df6afa08d52f6ea1bb6a35d394f833bbd756715d15cdd5a8bcf5c8ac
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=8efcb4d1d2fd38324c09615c4834a673ab9d2370fa4c9efea873a375e89f2bc0
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta
from datetime import datetime

# Load the historical data
df = pd.read_csv('/content/BTC_2019_2023_1d.csv')  # Replace with your CSV file path

df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')

# Handle invalid dates (optional)
df = df.dropna(subset=['datetime'])

# Set 'datetime' as index
df.set_index('datetime', inplace=True)

# Calculate Heikin Ashi Candles
df['HA_close'] = (df['open'] + df['high'] + df['low'] + df['close']) / 4
df['HA_open'] = (df['open'].shift(1) + df['close'].shift(1)) / 2
df['HA_high'] = df[['high', 'HA_open', 'HA_close']].max(axis=1)
df['HA_low'] = df[['low', 'HA_open', 'HA_close']].min(axis=1)

# Calculate technical indicators using the 'ta' library
df['RSI'] = ta.momentum.RSIIndicator(df['HA_close'], window=14).rsi()
macd = ta.trend.MACD(df['HA_close'], window_slow=26, window_fast=12, window_sign=9)
df['MACD'] = macd.macd()
df['MACD_Signal'] = macd.macd_signal()
df['MACD_Hist'] = macd.macd_diff()
df['ema_05'] = ta.trend.EMAIndicator(df['HA_close'], window=5).ema_indicator()
df['ema_10'] = ta.trend.EMAIndicator(df['HA_close'], window=10).ema_indicator()
df['ema_30'] = ta.trend.EMAIndicator(df['HA_close'], window=30).ema_indicator()
df['ATR'] = ta.volatility.AverageTrueRange(df['HA_high'], df['HA_low'], df['HA_close'], window=14).average_true_range()

# Initialize trading parameters
rsi_buy_threshold = 30  # Adjusted buy threshold for better entry
rsi_sell_threshold = 60  # Adjusted sell threshold for better exit
trailing_stop_multiplier = 2  # Multiplier for ATR in stop-loss
transaction_cost = 0.00

# Initialize variables
in_position = False
position_type = 0  # 1 for long, -1 for short
entry_price = 0
highest_price = 0
lowest_price = float('inf')
net_profit = 0
balance = 10000
btc_amount = 0
trade_count = 0
winning_trades = 0
losing_trades = 0
max_drawdown_value = 0
peak_balance = balance
signal_history = [0] * len(df)
balance_history = []

# Loop through DataFrame row by row
for i, (index, row) in enumerate(df.iterrows()):
    current_price = row['close']
    rsi = row['RSI']
    macd_value = row['MACD']
    macd_signal = row['MACD_Signal']
    ema_10 = row['ema_10']
    volume = row['volume']

    # Volume threshold for entering trades (e.g., average volume over last 10 days)
    volume_threshold = df['volume'].rolling(window=10).mean().iloc[i]

    # Buy condition: Strong bullish signal based on RSI, MACD, EMA, and volume
    if (rsi > rsi_buy_threshold and ema_10 > row['ema_30'] and macd_value > macd_signal and
        current_price > ema_10 and volume > volume_threshold and not in_position):
        # Enter long position
        entry_price = current_price
        btc_amount = (balance * (1 - transaction_cost)) / entry_price
        highest_price = entry_price
        in_position = True
        position_type = 1
        trade_count += 1
        signal_history[i] = 1  # Buy signal

    # Sell condition for long: Stop-loss based on trailing stop or strong bearish signals
    elif in_position and position_type == 1:
        if current_price > highest_price:
            highest_price = current_price

        # Dynamic stop-loss based on ATR
        stop_loss_price = entry_price - (trailing_stop_multiplier * row['ATR'])

        if current_price <= stop_loss_price or (rsi < rsi_sell_threshold or macd_value < macd_signal):
            # Exit long position
            balance = btc_amount * current_price * (1 - transaction_cost)
            profit = (current_price - entry_price) * btc_amount
            net_profit += profit
            if profit > 0:
                winning_trades += 1
            else:
                losing_trades += 1
            in_position = False
            signal_history[i] = -1  # Sell signal

    # Record balance history
    balance_history.append(balance)

    # Max drawdown tracking
    peak_balance = max(peak_balance, balance)
    drawdown = (peak_balance - balance) / peak_balance
    max_drawdown_value = max(max_drawdown_value, drawdown)


# Define trade type based on signals
trade_type = []
for signal in signal_history:
    if signal == 1:
        trade_type.append("Buy")
    elif signal == -1:
        trade_type.append("Sell")
    else:
        trade_type.append("Hold")

# Add signal and trade_type columns to the DataFrame
df['signals'] = signal_history
df['trade_type'] = trade_type

# Save the DataFrame to a CSV file with all original columns and the new columns
current_time = datetime.now().strftime("%Y-%m-%d")
output_filename = f'Strategy5BTCSharpe9.csv'
df.to_csv(output_filename, index=True)  # Include the index for datetime reference

print(f"File saved as {output_filename}")



# Function to calculate maximum drawdown
def calculate_max_drawdown(equity_curve):
    peak = equity_curve[0]
    max_drawdown = 0
    for x in equity_curve:
        if x > peak:
            peak = x
        drawdown = (peak - x) / peak
        if drawdown > max_drawdown:
            max_drawdown = drawdown
    return max_drawdown

File saved as Strategy5BTCSharpe9.csv


In [36]:
csv_file_path = "/content/Strategy5BTCSharpe9.csv"

In [37]:
!git clone https://github.com/ztuntrade/untrade-sdk.git
!pip install ./untrade-sdk/.

import uuid
import os
from untrade.client import Client

fatal: destination path 'untrade-sdk' already exists and is not an empty directory.
Processing ./untrade-sdk
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for untrade: filename=untrade-0.1.0-py3-none-any.whl size=5082 sha256=31197b3357f64b30ee36b40c8baf39aa49051fa7e417fb9f79f49806376b6a57
  Stored in directory: /root/.cache/pip/wheels/c1/ce/9c/59addc63ec0e1b26a267abbcf20bcb13f36bc7f128da6e1cd6
Successfully built untrade
  Attempting uninstall: untrade
    Found existing installation: untrade 0.1.0
    Uninstalling untrade-0.1.0:
      Successfully uninstalled untrade-0.1.0


In [38]:
def perform_backtest(csv_file_path):
     client = Client()
     result = client.backtest(
         jupyter_id="test",
         file_path=csv_file_path,
         leverage=1,
     )
     return result


def perform_backtest_large_csv(csv_file_path):
    client = Client()
    file_id = str(uuid.uuid4())
    chunk_size = 90 * 1024 * 1024  # 90 MB
    total_size = os.path.getsize(csv_file_path)
    total_chunks = (total_size + chunk_size - 1) // chunk_size
    chunk_number = 0

    # Handle small files
    if total_size <= chunk_size:
        total_chunks = 1
        result = client.backtest(
            file_path=csv_file_path,
            leverage=1,
            jupyter_id="test",
        )
        for value in result:
            print(value)
        return result

    # Process large files in chunks
    with open(csv_file_path, "rb") as f:
        while True:
            chunk_data = f.read(chunk_size)
            if not chunk_data:
                break

            # Save each chunk temporarily in /tmp
            chunk_file_path = f"/tmp/{file_id}_chunk{chunk_number}.csv"
            with open(chunk_file_path, "wb") as chunk_file:
                chunk_file.write(chunk_data)

            # Perform backtest on the current chunk
            result = client.backtest(
                file_path=chunk_file_path,
                leverage=1,
                jupyter_id="test",
                file_id=file_id,
                chunk_number=chunk_number,
                total_chunks=total_chunks,
            )

            # Process the results of the backtest
            for value in result:
                print(value)

            # Remove the temporary chunk file
            os.remove(chunk_file_path)

            # Move to the next chunk
            chunk_number += 1

    return result

In [39]:
#change to perform_backtest_large_csv(csv_file_path) for large files
backtest_result = perform_backtest(csv_file_path)
print(backtest_result)
for value in backtest_result:
    print(value)

<generator object Client._handle_response_stream at 0x7c66a9348430>
data: {
  "jupyter_id": "test",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-09-08 00:00:00",
      "Total Trades": 34,
      "Leverage Applied": 1.0,
      "Winning Trades": 19,
      "Losing Trades": 15,
      "No. of Long Trades": 34,
      "No. of Short Trades": 0,
      "Benchmark Return(%)": 325.632937,
      "Benchmark Return(on $1000)": 3256.329373,
      "Win Rate": 55.882353,
      "Winning Streak": 10,
      "Losing Streak": 6,
      "Gross Profit": 2706.804203,
      "Net Profit": 2655.804203,
      "Average Profit": 78.111888,
      "Maximum Drawdown(%)": 5.024687,
      "Average Drawdown(%)": 1.149131,
      "Largest Win": 638.552873,
      "Average Win": 163.224503,
      "Largest Loss": -68.876529,
      "Average Loss": -29.697424,
      "Maximum Holding Time": "47 days 0:0:0",
      "Average Holding Time": "11 days 2:7:3",
    